<a href="https://colab.research.google.com/github/dharsha454/Test-documents/blob/main/opencv_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=75d5e45a13e34556ce8950365ae5d959b76021a8364a210690d6f1bd7d44c674
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract


In [ ]:
import cv2
import numpy as np
import pytesseract
import os

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
imgQ = cv2.imread('Query.PNG')
cv2_imshow( imgQ)
cv2.waitKey(0)

In [ ]:
h,w,c = imgQ.shape
imgQ = cv2.resize(imgQ, (w//3, h//3))

In [ ]:
per = 25
# threshold pixels
#pixelthreshold = np.zeros00

In [ ]:
roi = [
[(156 ,  50),(277 ,  73), 'text', 'Height in inches'],
[(424 ,  48),(563 ,  74), 'text', 'Weight in pounds'],
[(161 ,  76),(387 ,  98), 'text', 'Blood Pressure'],
[(448 ,  79),(631 ,  100),'text', 'Pulse'],
[(184 ,  335),(288 ,  362), 'box', 'eyes normal'],
[(293 ,  334),(419 ,  365), 'box', 'eyes abnormal'],
[(185 ,  368),(289 ,  398) ,'box', 'ears normal'],
[(294 ,  367),(420 ,  399),'box', 'ears normal'],
]

In [ ]:
orb = cv2.ORB_create(1000)

In [ ]:
kp1, des1 = orb.detectAndCompute(imgQ, None)


In [ ]:
img = cv2.imread("TEST.PNG")
	#cv2.imshow(y,img)
kp2, des2 = orb.detectAndCompute(img, None)
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = bf.match(des2, des1)
matches.sort(key = lambda x: x.distance)
good = matches[:len(matches)*(per//100)]


In [ ]:
	#adjust number in good
	imgMatch = cv2.drawMatches(img, kp2, imgQ, kp1, good[:100], None, flags=2)
	cv2_imshow(imgMatch)


In [ ]:
srcPoints = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
dstPoints = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)



In [ ]:
M, _ = cv2.findHomography(srcPoints, dstPoints, cv2.RANSAC, 5.0)
imgScan = cv2.warpPerspective(img, M,(w,h))
imgScan = cv2.resize(imgScan, (w//3, h//3))
cv2_imshow(imgScan)

	


error: ignored

In [ ]:
imgShow = imgScan.copy()
	imgMask = np.zeros_like(imgShow)

	myData = []

	print(f'################ Extracting data from Form{j} ############### ')
	for x,r in enumerate(roi):

		cv2.rectangle(imgMask, (r[0][0], r[0][1]), (r[1][0], r[1][1]), (0,255,0), cv2.FILLED)
		imgShow = cv2.addWeighted(imgShow, 0.99, imgMask, 0.1, 0)

		imgCrop = imgScan[r[0][1]:r[1][1], r[0][0]:r[1][0]]
		#cv2.imshow(str(x), imgCrop)

		if r[2] =='text':
			print(f'{r[3]}:{pytesseract.image_to_string(imgCrop)}')
			myData.append(pytesseract.image_to_string(imgCrop))
		if r[2] == 'box':
			imgGray = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2GRAY)
			imgThresh = cv2.threshold(imgGray, 170,255, cv2.THRESH_BINARY_INV)[1]
			totalPixels = cv2.countNonZero(imgThresh)
			# count pixels to make decision for checked or unchecked and find a threshold for the same
			if totalPixels > pixelthreshold: totalPixels = 1;
			else: totalPixels =0
			print(f'{r[3]}:{totalPixels}')
			myData.append(totalPixels)
		#cv2.putText(imgShow, str(myData))